In [ ]:
!pip install transformers
!pip install tensorflow_addons

     |████████████████████████████████| 3.5 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 49.3 MB/s 
     |████████████████████████████████| 596 kB 60.3 MB/s 
     |████████████████████████████████| 6.8 MB 47.5 MB/s 
     |████████████████████████████████| 67 kB 3.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 5.5 MB/s 


In [ ]:
from google.colab import drive
import pickle
from transformers import BertTokenizer, TFAutoModel, AlbertTokenizerFast, TFBertModel
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

drive_loc = './drive/MyDrive/'

df = pd.read_csv(f'{drive_loc}/Applied.csv')

In [ ]:
model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name)

max_len = 512

entity_diction = { "Reason":2,
  "Defendant":1,
  "Penalties":3,
  "Outcome":4,
  "Platiff":0
}


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:

#example = training_data[2]
def labeling_to_output(train):

  text = train[0]
  tokens = train[1]
  stringer = tokenizer.tokenize(text)
  tok = tokenizer(
                  text,
                  max_length = max_len, #change this for padding
                  truncation=True,
                  padding='max_length',
                  add_special_tokens=True,
                  return_tensors='np'

  )

  output = np.zeros((len(tok['attention_mask'][0]), len(entity_diction)*3))
  
  for tag in tokens['entities']:
    start = tag[0]
    end = tag[1]
    tager = tag[2]

    before = tokenizer(text[:start], return_tensors='np')
    after = tokenizer(text[start:+end], return_tensors='np')

    starter = len(before['input_ids'][0]) - 1
    afterer = len(after['input_ids'][0]) - 1
    if (starter+afterer) < max_len:
      output[starter:(starter+afterer-1), entity_diction[tager]] = 1
      output[starter, (entity_diction[tager]+len(entity_diction))] = 1
      output[(starter+afterer-2), int((entity_diction[tager]+len(entity_diction)*2))] = 1
    

  return tok, output

In [ ]:
X = {'input_ids':[],
     'attention_mask':[],
     'token_type_ids':[],
     }
    
y = []


In [ ]:
for i in range(len(df)):
    text = df['Article'].values[i]
    example = (text, {'entities':[]})
    tok, output = labeling_to_output(example)
    inputer = tok['input_ids'].tolist()[0]
    mask = tok['attention_mask'].tolist()[0]
    types = tok['token_type_ids'].tolist()[0]
    X['input_ids'].append(inputer)
    X['attention_mask'].append(mask)
    X['token_type_ids'].append(types)
    y.append(output.tolist())


In [ ]:
X['input_ids'] = np.array(X['input_ids'])
X['attention_mask'] = np.array(X['attention_mask'])
X['token_type_ids'] = np.array(X['token_type_ids'])
y = np.array(y)
#y = np.array(y).reshape(len(y), int(y.shape[1]*y.shape[2]))

In [ ]:
from tensorflow.keras.layers import Dropout, Reshape, Softmax

dense_shape = 6

seq_len = max_len
check = 'checkpoint-InformationExtraction'
checkpoint_dir = f'{drive_loc}/BERT_NER'

checkpoint = ModelCheckpoint(checkpoint_dir, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')#, save_weights_only=True, mode='auto')
callbacks = [checkpoint]

def create_model():
  BERT = TFBertModel.from_pretrained(model_name)

  input_ids = tf.keras.layers.Input(shape=(seq_len, ), name='input_ids', dtype='int32')
  token_type_ids = tf.keras.layers.Input(shape=(seq_len, ), name='token_type_ids', dtype='int32')
  mask = tf.keras.layers.Input(shape=(seq_len, ), name='attention_mask', dtype='int32')

  embeddings = BERT(input_ids, token_type_ids=token_type_ids, attention_mask=mask)[0]#'pooler_output'] #Try out index 0
  #embeddings = BERT.albert(input_ids, attention_mask=mask)['pooler_output']

  #den_1 = tf.keras.layers.Dense(1000, activation='relu')(embeddings)]
  den_1 = Dropout(0.5)(embeddings)
  #den_2  = tf.keras.layers.Dense(1000, activation='relu')(den_1)
  #den_3 = Dropout(0.5)(den_2)
  #den_1 = tf.keras.layers.Dense(10000, activation='relu')(den_1)
  #den_1 = Dropout(0.5)(den_1)
  #den_1 = tf.keras.layers.Dense(1000, activation='relu')(den_1)
  output = tf.keras.layers.Dense(dense_shape)(den_1) #Alter this depending on output type
  output = Reshape((512, 2, 3))(output)#(512, 2, 3)
  output = Softmax()(output)

  model = tf.keras.Model(inputs=[input_ids, token_type_ids, mask],
                         outputs=output)

  #opt = tfa.optimizers.AdamW()
  opt = tf.keras.optimizers.Adam(learning_rate = 1e-5)
  #loss = tf.keras.losses.CategoricalCrossentropy()

  model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

  #model.load_weights(checkpoint_dir)
  model.layers[0].trainable = False
  model.layers[1].trainable = False
  model.layers[2].trainable = False
  model.layers[3].trainable = False

  return model

In [ ]:
from tensorflow.keras.layers import Dropout, Reshape, Softmax, Permute

def create_model_2():
  BERT = TFBertModel.from_pretrained(model_name)

  input_ids = tf.keras.layers.Input(shape=(seq_len, ), name='input_ids', dtype='int32')
  token_type_ids = tf.keras.layers.Input(shape=(seq_len, ), name='token_type_ids', dtype='int32')
  mask = tf.keras.layers.Input(shape=(seq_len, ), name='attention_mask', dtype='int32')

  embeddings = BERT(input_ids, token_type_ids=token_type_ids, attention_mask=mask)[0]#'pooler_output'] #Try out index 0
  #embeddings = BERT.albert(input_ids, attention_mask=mask)['pooler_output']
  dropout = Dropout(0.5)(embeddings)
  den_1 = tf.keras.layers.Dense(10000, activation = 'relu')(dropout)
  den_2 = tf.keras.layers.Dense(1000, activation = 'relu')
  output = tf.keras.layers.Dense(4)(dropout) #Alter this depending on output type
  output = Permute((2,1), input_shape=(512, ))(output)
  output = Softmax()(output)

  model = tf.keras.Model(inputs=[input_ids, token_type_ids, mask],
                         outputs=output)

  #opt = tfa.optimizers.AdamW()
  opt = tf.keras.optimizers.Adam(learning_rate = 1e-5)
  #loss = tf.keras.losses.CategoricalCrossentropy()

  model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

  #model.load_weights(checkpoint_dir)
  #model.layers[0].trainable = False
  #model.layers[1].trainable = False
  #model.layers[2].trainable = False
  #model.layers[3].trainable = False

  return model

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

# Create model
with strategy.scope():
    model = create_model()
    model_2 = create_model_2()
    model.load_weights(f'{checkpoint_dir}.h5')
    model_2.load_weights(f'{checkpoint_dir}_2.h5')
    _model_ = tf.keras.Model(model_2.input, model_2.get_layer('permute').output)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.0.46.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.0.46.186:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are in

In [ ]:
import copy

def shortlist_individual(values, cutoff, length):
  zeros = np.zeros(len(values))
  for i in range(len(values)):
    val = values[i]
    if val > cutoff:
      start = i - length
      end = i + length
      if start < 0:
        start = 0
      else:
        if end >= len(values):
          end = len(values)-1
      zeros[start:end] = 1
  return zeros

def shortlist_individual_argmax(_values_, indexer, cutoff, length):
  
  values = copy.copy(_values_)

  for i in range(len(_values_)):
    zeros = np.zeros(_values_.shape[-1])
    argmaxer = _values_[i].argmax()
    zeros[argmaxer] = 1
    values[i] = zeros
  values = values[:, indexer]
  zeros = np.zeros(len(values))
  for i in range(len(values)):
    val = values[i]
    if val > cutoff:
      start = i - length
      end = i + length
      if start < 0:
        start = 0
      else:
        if end >= len(values):
          end = len(values)-1
      zeros[start:end] = 1
  return zeros, values

def evaluate_phrases(_X_, values, values_finish, averages, cutoff):

  phrase_coordinates = []

  last_val = 0
  current_coor = []

  #Find Phrase Coordinates
  for i in range(len(values)):
    val = values[i]
    if val != 0:
      if last_val == 0:
        current_coor.append(i)
        current_coor.append(i)
      else:
        current_coor[1] = i
    else:
      if last_val != 0:
        phrase_coordinates.append(current_coor)
        current_coor = []
    last_val = val

  #Find start finish of Sentance
  extracts = []
  for coor in phrase_coordinates:
    st = coor[0]
    en = coor[1]

    starting_vals = values[st:en]
    ending_vals = values_finish[st:en]

    arg_st = starting_vals.argmax()
    arg_en = ending_vals.argmax()

    _st_ = st+arg_st
    _en_ = st+arg_en+1
    
    if _st_ <= en:
      mean = averages[_st_:_en_].mean()
      if mean > cutoff:
        extracts.append(_X_[_st_:_en_])
  
  #Convert Tokens to Text
  texts = []
  for extract in extracts:
    texts.append(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(extract)))

  return texts

def get_entities_argmax_altered(text, out, prob):
  
  new_out = []

  for i in range(len(out)):
      new = np.zeros(len(out[i]))
      maxer = out[i].argmax()
      new[maxer] = 1
      new_out.append(new.tolist())
  
  output = np.array(new_out)
  
  identified = []

  entity_loc = { 
    0:["Plantiff", False, []],
    1:["Defendant",False, []],
  }
  
  for i in range(1, len(output)):
      word = text[i]
      for n in range(2):
          open = entity_loc[n][1]
          try:
            val = output[i, n]
          except:
            pdb.set_trace()
          if open == True:
              if val == 1:
                  entity_loc[n][2].append(word)
              else:
                  entity_loc[n][1] = False
                  identified.append([entity_loc[n][0], tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(entity_loc[n][2]))])
                  entity_loc[n][2] = []
          else:
              if val == 1:
                  entity_loc[n][2].append(word)
                  entity_loc[n][1] = True

  return identified

In [ ]:
predictions_1 = model.predict(X)

Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.


In [ ]:
_pred_ = _model_.predict(X)

In [ ]:
type_diction = {
    'Reason':[[1, 0], [0, 2]],
    'Outcome':[[1, 1], [1, 3]]
    }

name_diction = []

identified_texts = {}

individuals = copy.copy(predictions_1)
start_stops = copy.copy(_pred_)

for key in type_diction.keys():
  for i in range(len(df)):
    individual = individuals[i]
    start_stop = start_stops[i]

    short, binary_vals = shortlist_individual_argmax(individual[:, 1], type_diction[key][0][1], 0.5, 6)
    
    values = short*start_stop[type_diction[key][1][0], :]
    values_finish = short*start_stop[type_diction[key][1][1], :]

    texts = evaluate_phrases(X['input_ids'][i], values, values_finish, binary_vals, 0.5)

    if i not in identified_texts:
      identified_texts[i] = {}
    
    identified_texts[i][key] = texts
    df[key].values[i] = texts

df['Entities'] = ''
for i in range(len(df)):
  entities = get_entities_argmax_altered(X['input_ids'][i], predictions_1[i, :, 0, :], 0.5)
  df['Entities'].values[i] = entities
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
df.to_csv(f'{drive_loc}/Applied_2.csv', index=False)